# Check coverage

In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print(repo_dir)
if repo_dir not in sys.path:
    sys.path = [repo_dir] + sys.path
else:
    sys.path.remove(repo_dir)
    sys.path = [repo_dir] + sys.path
    


/Users/marijnkoolen/Code/Huygens/republic-project


In [2]:
from elasticsearch import Elasticsearch
import json

from republic.elastic.republic_elasticsearch import initialize_es

base_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/"

rep_es = initialize_es(host_type="external")


In [4]:
import pandas as pd
from do_indexing import get_preprocessed_pages, get_raw_pages, Indexer
from republic.model.inventory_mapping import read_inventory_metadata
from republic.helper.metadata_helper import get_per_page_type_index
from republic.model.inventory_mapping import get_inventory_by_num


indexer = Indexer(host_type="external", base_dir='../../data')

rep_es.config['session_metadata_index'] = 'session_metadata_staging'
rep_es.config['session_text_region_index'] = 'session_text_regions_staging'
rep_es.config['resolutions_index'] = 'full_resolutions_staging'


W0211 21:30:08.624000 3021 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/opt/homebrew/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: /opt/homebrew/lib/python3.11/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [112]:
for inv_num in [3259, 3319]:
    pages = list(get_raw_pages(inv_num, indexer))
    inv_meta = get_inventory_by_num(inv_num)
    page_type_index = get_per_page_type_index(inv_meta)
    page_type_index
    for page in pages:
        if page.has_type('empty_page'):
            page.remove_type('empty_page')
        pt = page_type_index[page.metadata['page_num']]
        if not page.has_type(pt):
            page.add_type(pt)
        page.metadata['type'] = [pt for pt in page.type]
    pages_file = f"../../data/pages/raw_page_json/preprocessed_pages-{inv_num}.jsonl.gz"
    write_pages(pages_file, pages)


raw_pages_file: ../../data/pages/raw_page_json/raw_pages-3259.jsonl.gz
Reading raw pages from file for inventory 3259
raw_pages_file: ../../data/pages/raw_page_json/raw_pages-3319.jsonl.gz
Reading raw pages from file for inventory 3319


In [111]:
len(pages)

2714

In [109]:
from do_indexing import write_pages
[pt for page in pages for pt in page.type if pt not in {'structure_doc', 'physical_structure_doc', 'text_region', 'pagexml_doc', 'page'}]



In [5]:

def get_page_lines(inv_num, indexer):
    pages = [page for page in get_preprocessed_pages(inv_num, indexer, use_line_class_update=True)]
    page_df = pd.DataFrame([{'page_id': page.id, 'line_id': line.id, 'text': line.text, 'lc': line.metadata['line_class']} for page in pages for line in page.get_lines()])
    return page_df



def get_session_lines(inv_num):
    session_metas = rep_es.retrieve_inventory_session_metadata(inv_num)
    session_trs = [tr for tr in rep_es.retrieve_inventory_session_text_regions(inv_num)]
    session_df = pd.DataFrame([{'session_id': tr.metadata['session_id'], 'line_id': line.id} for tr in session_trs for line in tr.lines])
    return session_df

def get_resolution_lines(inv_num):
    query = {'match': {'metadata.inventory_num': inv_num}}
    resolutions = list(rep_es.scroll_resolutions_by_query(query, size=10))
    rows = []
    for res in resolutions:
        trs = [tr for para in res.paragraphs for tr in para.text_regions]
        trs.extend([tr for tr in res.linked_text_regions])
        for tr in trs:
            for line in tr.lines:
                row = {'resolution_id': res.id, 'line_id': line.id}
                rows.append(row)
    resolution_df = pd.DataFrame(rows)
    return resolution_df


indexer = Indexer(host_type="external", base_dir='../../data')

missing_dir = '../../data/sessions/missing_lines'
if not os.path.exists(missing_dir):
    os.mkdir(missing_dir)

invs_meta = read_inventory_metadata()
inv_sessions = {}


## Making missing line info

In [37]:

for inv_meta in invs_meta:
    inv_num = inv_meta['inventory_num']
    if inv_num < 3097:
        continue
    page_df = get_page_lines(inv_num, indexer)
    if len(page_df) == 0:
        continue
    session_df = get_session_lines(inv_num)
    resolution_df = get_resolution_lines(inv_num)
    merge_df = pd.merge(page_df, session_df, on='line_id', how='left')
    merge_df = pd.merge(merge_df, resolution_df, on='line_id', how='left')
    missing_df = merge_df[merge_df.session_id.isna()]
    line_file = os.path.join(missing_dir, f"inv_lines-inv-{inv_num}.tsv.gz")
    merge_df.to_csv(line_file, sep='\t', index=False, compression='gzip')
    # query = {'match': {'metadata.inventory_num': inv_num}}
    # response = rep_es.es_anno.search(query=query, index='session_metadata_staging', size=0)
    # total = response['hits']['total']['value']
    # inv_session[inv_num] = total

preprocessed_pages_file: ../../data/pages/line_class_preprocessed_page_json/preprocessed_pages-3097.jsonl.gz
Reading preprocessed pages from file for inventory 3097
total hits:      184	hits per scroll: 10
total hits:     7454	hits per scroll: 100
total hits:     1705	hits per scroll: 10
preprocessed_pages_file: ../../data/pages/preprocessed_page_json/preprocessed_pages-3098.jsonl.gz
preprocessed_pages_file: ../../data/pages/line_class_preprocessed_page_json/preprocessed_pages-3099.jsonl.gz
Reading preprocessed pages from file for inventory 3099
total hits:      125	hits per scroll: 10
total hits:     5980	hits per scroll: 100
total hits:     1432	hits per scroll: 10
preprocessed_pages_file: ../../data/pages/line_class_preprocessed_page_json/preprocessed_pages-3102.jsonl.gz
Reading preprocessed pages from file for inventory 3102
total hits:      362	hits per scroll: 10
total hits:     6306	hits per scroll: 100
total hits:     1152	hits per scroll: 10
preprocessed_pages_file: ../../data

## Updating missing line info

In [6]:
from pathlib import Path

missing_dir = Path('../../data/sessions/missing_lines')
missing_files = list(missing_dir.glob('missing*'))
line_files = list(missing_dir.glob('inv*'))
len(missing_files), len(line_files)

(382, 382)

In [7]:
line_file_map = {int(lf.name.split('-inv-')[-1].replace('.tsv.gz', '')): lf for lf in line_files}


In [174]:

for inv_meta in invs_meta:
    inv_num = inv_meta['inventory_num']
    if 400 <= inv_num <= 454:
        continue
    if inv_num < 3217:
        continue
    if inv_num not in {3296,3230,4548}:
        continue
    if inv_num not in line_file_map:
        print(f"missing inventory: {inv_num}")
        continue
    line_file = line_file_map[inv_num]
    inv_df = pd.read_csv(line_file, sep='\t', compression='gzip')
    resolution_df = get_resolution_lines(inv_num)
    updated_df = pd.merge(inv_df.drop('resolution_id', axis=1), resolution_df, on='line_id', how='left')
    orig_res_lines = inv_df[inv_df.resolution_id.notna()].shape
    update_res_lines = updated_df[updated_df.resolution_id.notna()].shape
    print(f"inv {inv_num}\torig lines: {orig_res_lines}\tupdated lines: {update_res_lines}")
    if update_res_lines > orig_res_lines:
        updated_df.to_csv(line_file, sep='\t', index=False, compression='gzip')
    

total hits:     1572	hits per scroll: 10
inv 3230	orig lines: (47462, 6)	updated lines: (53512, 6)
total hits:     1953	hits per scroll: 10
inv 3296	orig lines: (48709, 6)	updated lines: (57822, 6)
total hits:      144	hits per scroll: 10
inv 4548	orig lines: (25246, 6)	updated lines: (35841, 6)


In [140]:
len(inv_df), len(resolution_df)

(31029, 28996)

In [147]:
resolution_df


((19279, 6), (28996, 6))

### Loading inventory line files

In [175]:
invs_df = []
for line_file in line_files:
    inv_df = pd.read_csv(line_file, sep='\t', compression='gzip')
    invs_df.append(inv_df)

inv_df = pd.concat(invs_df)



In [176]:
inv_df['inventory_id'] = inv_df.page_id.apply(lambda x: x.split('_')[2])
inv_df['page_num'] = inv_df.page_id.apply(lambda x: int(x.split('-page-')[-1]))
inv_df['page_side'] = inv_df.page_num.apply(lambda x: 'verso' if x % 2 == 0 else 'recto')
inv_df['line_left'] = inv_df.line_id.apply(lambda x: int(x.split('-line-')[-1].split('-')[0]))


In [177]:
inv_df[inv_df.resolution_id.apply(lambda x: pd.notna(x) and 'attendance' in x)]

,page_id,line_id,text,lc,session_id,resolution_id,inventory_id,page_num,page_side,line_left
2,NL-HaNA_1.01.02_4808_0004-page-7,NL-HaNA_1.01.02_4808_0004-line-2732-872-892-218,preside d'Ers. van Gent.,attendance,session-4808-num-1,session-4808-num-1-attendance_list,4808,7,recto,2732
68,NL-HaNA_1.01.02_4808_0005-page-9,NL-HaNA_1.01.02_4808_0005-line-2591-566-116-81,58e,table,session-4808-num-1,session-4808-num-1-attendance_list,4808,9,recto,2591
69,NL-HaNA_1.01.02_4808_0005-page-9,NL-HaNA_1.01.02_4808_0005-line-2624-686-211-81,008,table,session-4808-num-1,session-4808-num-1-attendance_list,4808,9,recto,2624
70,NL-HaNA_1.01.02_4808_0005-page-9,NL-HaNA_1.01.02_4808_0005-line-2572-1823-179-78,54.,table,session-4808-num-1,session-4808-num-1-attendance_list,4808,9,recto,2572
96,NL-HaNA_1.01.02_4808_0005-page-9,NL-HaNA_1.01.02_4808_0005-line-2725-2488-451-74,v 29:de,attendance,session-4808-num-1,session-4808-num-1-attendance_list,4808,9,recto,2725
...,...,...,...,...,...,...,...,...,...,...
26994,NL-HaNA_1.01.02_4854_0544-page-1087,NL-HaNA_1.01.02_4854_0544-line-3116-2847-1738-177,im Reich und in ihre Kaijss. Mrs. dienst,attendance,session-4854-num-164,session-4854-num-164-attendance_list,4854,1087,recto,3116
27187,NL-HaNA_1.01.02_4854_0548-page-1095,NL-HaNA_1.01.02_4854_0548-line-4132-2869-845-141,der Cronen satisfactiones,attendance,session-4854-num-164,session-4854-num-164-attendance_list,4854,1095,recto,4132
27577,NL-HaNA_1.01.02_4854_0557-page-1112,NL-HaNA_1.01.02_4854_0557-line-864-2508-365-138,sij hun,attendance,session-4854-num-164,session-4854-num-164-attendance_list,4854,1112,verso,864
27939,NL-HaNA_1.01.02_4854_0564-page-1126,NL-HaNA_1.01.02_4854_0564-line-765-839-1684-199,"De Knuijt, G: van Rheede, F. V. Doma,",attendance,session-4854-num-167,session-4854-num-167-attendance_list,4854,1126,verso,765


In [178]:
att_df = inv_df[inv_df.lc == 'attendance']

In [179]:
import numpy as np
att_df.groupby('page_side').line_left.describe(percentiles=np.arange(0.1, 1.0, 0.1))

,count,mean,std,min,10%,20%,30%,40%,50%,60%,70%,80%,90%,max
page_side,,,,,,,,,,,,,,
recto,248636.0,2963.407121,457.638404,75.0,2571.0,2650.0,2723.0,2801.0,2879.0,2957.0,3037.0,3155.0,3439.0,13019.0
verso,245646.0,486.954650,395.072268,22.0,186.0,237.0,281.0,324.0,368.0,419.0,488.0,625.0,942.0,4943.0


In [180]:
page_not_session = inv_df[(inv_df.page_id.notna()) & (inv_df.session_id.isna())]
page_not_session.page_id.value_counts().head(20)

page_id
NL-HaNA_1.01.02_4771_0705-page-1409    148
NL-HaNA_1.01.02_3259_0052-page-102     124
NL-HaNA_1.01.02_3240_0060-page-119     123
NL-HaNA_1.01.02_3240_0060-page-118     118
NL-HaNA_1.01.02_4548_0128-page-254     106
NL-HaNA_1.01.02_3259_0037-page-72      105
NL-HaNA_1.01.02_3218_0001-page-0       105
NL-HaNA_1.01.02_3231_0339-page-676     104
NL-HaNA_1.01.02_3259_0033-page-64      101
NL-HaNA_1.01.02_4860_0176-page-350     100
NL-HaNA_1.01.02_3259_0047-page-92       98
NL-HaNA_1.01.02_3269_0152-page-303      93
NL-HaNA_1.01.02_4618_0450-page-899      93
NL-HaNA_1.01.02_4566_0309-page-617      93
NL-HaNA_1.01.02_4813_0558-page-1115     88
NL-HaNA_1.01.02_4634_0250-page-498      87
NL-HaNA_1.01.02_3259_0065-page-129      86
NL-HaNA_1.01.02_3259_0058-page-114      86
NL-HaNA_1.01.02_4573_0269-page-536      85
NL-HaNA_1.01.02_3259_0051-page-100      84
Name: count, dtype: int64

In [181]:
page_not_session[page_not_session.lc.isin(['date_header', 'empty', 'noise']) == False].inventory_id.value_counts()

inventory_id
3259    7120
3319    4380
3099    2443
4674    1819
4791    1302
        ... 
4759      38
3303      35
3106      27
4649      21
4822       6
Name: count, Length: 382, dtype: int64

In [182]:
page_not_session[page_not_session.page_id.str.contains('_3259_')].page_id.value_counts().head(30)
page_not_session[page_not_session.page_id.str.contains('_3259_')].lc.value_counts()


lc
table             4489
empty             1017
unknown            885
date_header        616
para_mid           604
noise              462
attendance         360
marginalia         241
para_start         191
date               182
para_end           167
insert_omitted       1
Name: count, dtype: int64

In [183]:
inv_df.shape, inv_df[inv_df.session_id.isna()].shape, inv_df[inv_df.resolution_id.isna()].shape

((17346603, 10), (337832, 10), (645144, 10))

In [184]:
page_not_session.lc.value_counts()

lc
date_header       81892
empty             81146
noise             62660
unknown           41055
marginalia        31602
attendance        15215
table             10210
para_mid           8376
para_start         3066
para_end           1081
date                915
superscript         292
insert_omitted      201
title               118
extract_intro         3
Name: count, dtype: int64

In [185]:
page_id = 'NL-HaNA_1.01.02_4808_0208-page-414'
page_id = 'NL-HaNA_1.01.02_3240_0060-page-119'
page_id = 'NL-HaNA_1.01.02_4548_0128-page-254'
page_not_session[page_not_session.page_id == page_id]


,page_id,line_id,text,lc,session_id,resolution_id,inventory_id,page_num,page_side,line_left
13686,NL-HaNA_1.01.02_4548_0128-page-254,NL-HaNA_1.01.02_4548_0128-line-121-77-942-138,Exhibitury den 29 Juny 1792,marginalia,NaN,NaN,4548,254,verso,121
13687,NL-HaNA_1.01.02_4548_0128-page-254,NL-HaNA_1.01.02_4548_0128-line-86-149-661-136,Secreet 8. 59. 8.,marginalia,NaN,NaN,4548,254,verso,86
13688,NL-HaNA_1.01.02_4548_0128-page-254,NL-HaNA_1.01.02_4548_0128-line-667-547-570-101,",giment van.",noise,NaN,NaN,4548,254,verso,667
13689,NL-HaNA_1.01.02_4548_0128-page-254,NL-HaNA_1.01.02_4548_0128-line-673-1077-65-57,NaN,empty,NaN,NaN,4548,254,verso,673
13690,NL-HaNA_1.01.02_4548_0128-page-254,NL-HaNA_1.01.02_4548_0128-line-649-1106-89-81,NaN,empty,NaN,NaN,4548,254,verso,649
...,...,...,...,...,...,...,...,...,...,...
13817,NL-HaNA_1.01.02_4548_0128-page-254,NL-HaNA_1.01.02_4548_0128-line-3966-1239-130-97,88,date_header,NaN,NaN,4548,254,verso,3966
13818,NL-HaNA_1.01.02_4548_0128-page-254,NL-HaNA_1.01.02_4548_0128-line-4089-1269-74-83,NaN,empty,NaN,NaN,4548,254,verso,4089
13819,NL-HaNA_1.01.02_4548_0128-page-254,NL-HaNA_1.01.02_4548_0128-line-4124-1253-76-94,NaN,empty,NaN,NaN,4548,254,verso,4124
13820,NL-HaNA_1.01.02_4548_0128-page-254,NL-HaNA_1.01.02_4548_0128-line-4184-1318-104-100,. gesse,marginalia,NaN,NaN,4548,254,verso,4184


In [186]:
session_not_resolution = inv_df[(inv_df.session_id.notna()) & (inv_df.resolution_id.isna())]
session_not_resolution.lc.value_counts()

lc
date              260172
attendance         26993
date_header         4280
marginalia          3937
noise               3362
table               2678
unknown             2626
para_mid            1361
para_end             568
empty                472
para_start           465
title                327
insert_omitted        67
superscript            4
Name: count, dtype: int64

In [187]:
session_not_resolution = inv_df[(inv_df.session_id.notna()) & (inv_df.resolution_id.isna())]
session_not_resolution.lc.value_counts()

lc
date              260172
attendance         26993
date_header         4280
marginalia          3937
noise               3362
table               2678
unknown             2626
para_mid            1361
para_end             568
empty                472
para_start           465
title                327
insert_omitted        67
superscript            4
Name: count, dtype: int64

In [188]:
content_lcs = ['para_mid', 'para_start', 'para_end', 'table', 'title']
session_not_resolution[session_not_resolution.lc.isin(content_lcs)].session_id.value_counts()

session_id
session-4566-num-266    365
session-4573-num-119    331
session-4542-num-80     260
session-3289-num-13     136
session-4614-num-268    115
                       ... 
session-3097-num-146      1
session-3097-num-142      1
session-3097-num-138      1
session-3097-num-137      1
session-4638-num-141      1
Name: count, Length: 1942, dtype: int64

In [189]:
s = session_not_resolution[session_not_resolution.lc.isin(content_lcs)].inventory_id.value_counts()
s

inventory_id
4573    406
4566    378
4542    276
3289    138
4614    134
       ... 
3222      1
3274      1
3205      1
3201      1
4808      1
Name: count, Length: 330, dtype: int64

In [191]:
','.join(s[s > 100].index)

'4573,4566,4542,3289,4614,3263,4548,4618,3102,3218'

In [192]:
session_not_resolution[(session_not_resolution.inventory_id == '3340') & (session_not_resolution.lc == 'para_start')]
session_not_resolution[(session_not_resolution.inventory_id == '3340') & (session_not_resolution.lc.isin(content_lcs))].session_id.value_counts()


Series([], Name: count, dtype: int64)

In [141]:
session_id = 'session-4813-num-1'
session_id = 'session-3289-num-43'
session_id = 'session-3270-num-233'
session_id = 'session-4831-num-12'
session_id = 'session-3340-num-148'
session_not_resolution[(session_not_resolution.session_id == session_id) & (session_not_resolution.lc != 'para_start')]

,page_id,line_id,text,lc,session_id,resolution_id,inventory_id
42656,NL-HaNA_1.01.02_3340_0557-page-1113,NL-HaNA_1.01.02_3340_0557-line-3161-1276-136-76,5.,para_end,session-3340-num-148,NaN,3340
42658,NL-HaNA_1.01.02_3340_0557-page-1113,NL-HaNA_1.01.02_3340_0557-line-3432-1171-151-141,P.,noise,session-3340-num-148,NaN,3340
42659,NL-HaNA_1.01.02_3340_0557-page-1113,NL-HaNA_1.01.02_3340_0557-line-3288-1206-1286-140,Jovis den 3. December,date,session-3340-num-148,NaN,3340
42660,NL-HaNA_1.01.02_3340_0557-page-1113,NL-HaNA_1.01.02_3340_0557-line-3632-1344-379-89,1699.,date,session-3340-num-148,NaN,3340
42822,NL-HaNA_1.01.02_3340_0559-page-1117,NL-HaNA_1.01.02_3340_0559-line-2702-3142-364-114,Mattele,marginalia,session-3340-num-148,NaN,3340
...,...,...,...,...,...,...,...
43672,NL-HaNA_1.01.02_3340_0570-page-1138,NL-HaNA_1.01.02_3340_0570-line-916-1978-1471-168,"heeren van Essen, ende andere hare Hoogh",para_mid,session-3340-num-148,NaN,3340
43673,NL-HaNA_1.01.02_3340_0570-page-1138,NL-HaNA_1.01.02_3340_0570-line-907-2086-1486-141,"Mogende Gedeputeerden, tot de buijten„",para_mid,session-3340-num-148,NaN,3340
43674,NL-HaNA_1.01.02_3340_0570-page-1138,NL-HaNA_1.01.02_3340_0570-line-924-2169-1573-194,"landsche zaken, om te visiteren, examineren,",para_mid,session-3340-num-148,NaN,3340
43675,NL-HaNA_1.01.02_3340_0570-page-1138,NL-HaNA_1.01.02_3340_0570-line-921-2295-1454-108,ende van alles alhier ter vergaderinge,para_mid,session-3340-num-148,NaN,3340


In [76]:
session_not_resolution[(session_not_resolution.session_id == session_id)].lc.value_counts()

lc
table          1060
attendance      488
para_mid         48
date             28
date_header       4
marginalia        2
para_end          2
Name: count, dtype: int64

In [33]:
import gzip
import json
import republic.model.republic_document_model as rdm



### Retrieving session from ES index

In [111]:
inv_num = 3270

session_id = 'session-3270-num-233'
session_id = 'session-4831-num-12'
retrieved_session = rep_es.retrieve_session_by_id(session_id)
retrieved_session.id, retrieved_session.stats

total hits:        1	hits per scroll: 1


('session-4831-num-12', {'lines': 2654, 'words': 6098, 'text_regions': 816})

In [64]:
for tr in retrieved_session.text_regions:
    print(f"{tr.id: <60}  {tr.metadata['page_id']: <40}  {tr.stats['lines']}")

NL-HaNA_1.01.02_4831_0011-text_region-4189-2236-1324-246      NL-HaNA_1.01.02_4831_0011-page-21         2
NL-HaNA_1.01.02_4831_0011-text_region-3049-2253-815-264       NL-HaNA_1.01.02_4831_0011-page-21         1
NL-HaNA_1.01.02_4831_0011-text_region-3700-2486-2072-1975     NL-HaNA_1.01.02_4831_0011-page-21         23
NL-HaNA_1.01.02_4831_0011-text_region-3065-2630-668-239       NL-HaNA_1.01.02_4831_0011-page-21         2
NL-HaNA_1.01.02_4831_0012-text_region-957-166-1972-4344       NL-HaNA_1.01.02_4831_0012-page-22         48
NL-HaNA_1.01.02_4831_0012-text_region-251-1384-721-133        NL-HaNA_1.01.02_4831_0012-page-22         1
NL-HaNA_1.01.02_4831_0012-text_region-3796-117-1906-448       NL-HaNA_1.01.02_4831_0012-page-23         4
NL-HaNA_1.01.02_4831_0012-text_region-3735-504-1124-692       NL-HaNA_1.01.02_4831_0012-page-23         5
NL-HaNA_1.01.02_4831_0012-text_region-3771-1176-1909-232      NL-HaNA_1.01.02_4831_0012-page-23         3
NL-HaNA_1.01.02_4831_0012-text_region-3713-1

### Loading session from dumped JSON

In [112]:
inv_num = 3270

session_file = '../../data/sessions/session_json/3270/session-3270-num-233.json.gz'
session_file = '../../data/sessions/session_json/4831/session-4831-num-12.json.gz'
with gzip.open(session_file, 'rt') as fh:
    session_json = json.load(fh)
    dumped_session = rdm.json_to_republic_session(session_json)

dumped_session.id, dumped_session.stats

('session-4831-num-12', {'lines': 2654, 'words': 6098, 'text_regions': 816})

In [115]:
for tr in dumped_session.text_regions:
    print(f"{tr.id: <60}  {tr.metadata['page_id']: <40}  {tr.stats['lines']: >4}    {tr.metadata['text_region_class']}")

NL-HaNA_1.01.02_4831_0011-text_region-4189-2236-1324-246      NL-HaNA_1.01.02_4831_0011-page-21            2    date
NL-HaNA_1.01.02_4831_0011-text_region-3049-2253-815-264       NL-HaNA_1.01.02_4831_0011-page-21            1    attendance
NL-HaNA_1.01.02_4831_0011-text_region-3700-2486-2072-1975     NL-HaNA_1.01.02_4831_0011-page-21           23    para
NL-HaNA_1.01.02_4831_0011-text_region-3065-2630-668-239       NL-HaNA_1.01.02_4831_0011-page-21            2    marginalia
NL-HaNA_1.01.02_4831_0012-text_region-957-166-1972-4344       NL-HaNA_1.01.02_4831_0012-page-22           48    para
NL-HaNA_1.01.02_4831_0012-text_region-251-1384-721-133        NL-HaNA_1.01.02_4831_0012-page-22            1    marginalia
NL-HaNA_1.01.02_4831_0012-text_region-3796-117-1906-448       NL-HaNA_1.01.02_4831_0012-page-23            4    para
NL-HaNA_1.01.02_4831_0012-text_region-3735-504-1124-692       NL-HaNA_1.01.02_4831_0012-page-23            5    para
NL-HaNA_1.01.02_4831_0012-text_region-3771-117

In [113]:
from do_indexing import make_opening_searcher
import republic.parser.logical.handwritten_resolution_parser as res_parser

inv_meta = get_inventory_by_num(inv_num)
opening_searcher = make_opening_searcher(inv_meta['year_start'], inv_meta['year_end'], debug=0)

resolutions = []
for res in res_parser.get_session_resolutions(retrieved_session, opening_searcher, debug=0):
    print(res.id, res.stats)
    resolutions.append(res)

tr: NL-HaNA_1.01.02_4831_0011-text_region-3049-2253-815-264 margin_left: 2800 tr.coords.left: 3049
tr: NL-HaNA_1.01.02_4831_0012-text_region-3713-1146-1150-2910 margin_left: 2800 tr.coords.left: 3713
tr: NL-HaNA_1.01.02_4831_0013-text_region-3744-233-1081-1851 margin_left: 2800 tr.coords.left: 3744
tr: NL-HaNA_1.01.02_4831_0013-text_region-3719-4052-920-349 margin_left: 2800 tr.coords.left: 3719
tr: NL-HaNA_1.01.02_4831_0014-text_region-1075-143-1102-1405 margin_left: 300 tr.coords.left: 1075
tr: NL-HaNA_1.01.02_4831_0014-text_region-1096-2109-875-222 margin_left: 300 tr.coords.left: 1096
tr: NL-HaNA_1.01.02_4831_0014-text_region-1103-3370-870-797 margin_left: 300 tr.coords.left: 1103
tr: NL-HaNA_1.01.02_4831_0014-text_region-4302-725-391-97 margin_left: 2800 tr.coords.left: 4302
tr: NL-HaNA_1.01.02_4831_0014-text_region-3681-2684-941-719 margin_left: 2800 tr.coords.left: 3681
tr: NL-HaNA_1.01.02_4831_0015-text_region-934-169-1079-1284 margin_left: 300 tr.coords.left: 934
tr: NL-HaNA_1

In [134]:
resolutions = []
session_file = '../../data/sessions/session_json/4831/session-4831-num-12.json.gz'
with gzip.open(session_file, 'rt') as fh:
    session_json = json.load(fh)
    dumped_session = rdm.json_to_republic_session(session_json)
    print(Counter([line.metadata['line_class'] for tr in dumped_session.text_regions for line in tr.lines]))

for res in res_parser.get_session_resolutions(dumped_session, opening_searcher, debug=0):
    print(res.id, res.stats)
    resolutions.append(res)

Counter({'table': 1889, 'attendance': 531, 'para_mid': 159, 'date': 35, 'para_end': 16, 'para_start': 10, 'extract_date': 8, 'marginalia': 5, 'title': 1})
session-4831-num-12-attendance_list {'lines': 40, 'words': 119, 'text_regions': 0, 'paragraphs': 1}
session-4831-num-12-resolution-1 {'lines': 2576, 'words': 5943, 'text_regions': 0, 'paragraphs': 24}


In [110]:
from collections import Counter
from republic.helper.pagexml_helper import make_baseline_string as mbs

for tr in dumped_session.text_regions:
    page_num = int(tr.metadata['page_id'].split('-page-')[-1])
    if page_num % 2 == 1:
        margin_left = 2800
    else:
        margin_left = 300
    if not tr.has_type('attendance'):
        continue
    if tr.coords.left - margin_left > 400:
        tr.add_type('table')
        tr.remove_type('attendance')
        for line in tr.lines:
            line.metadata['line_class'] == 'table'
    # for line in tr.lines:
    #     print(f"    {mbs(line)}  {line.text}")
    #print(tr.metadata)
    print(tr.type, tr.coords.left, Counter([line.metadata['line_class'] for line in tr.lines]))
    #break
    

['text_region', 'attendance'] 3049 Counter({'attendance': 1})
['text_region', 'attendance'] 341 Counter({'attendance': 9})
['text_region', 'attendance'] 275 Counter({'attendance': 17})
['text_region', 'attendance'] 3073 Counter({'attendance': 1})


In [26]:
sum_lines = 0
for res in resolutions:
    print(f"------------\n")
    lines = [line for para in res.paragraphs for tr in para.text_regions for line in tr.lines]
    sum_lines += len(lines)
    print(f"res: {res.id: <40}  num lines: {len(lines): >6}  sum lines: {sum_lines: >6}")

------------

res: session-3270-num-233-attendance_list      num lines:      6  sum lines:      6
------------

res: session-3270-num-233-resolution-1         num lines:      4  sum lines:     10
------------

res: session-3270-num-233-resolution-2         num lines:     44  sum lines:     54
------------

res: session-3270-num-233-resolution-3         num lines:     97  sum lines:    151
------------

res: session-3270-num-233-resolution-4         num lines:     21  sum lines:    172
------------

res: session-3270-num-233-resolution-5         num lines:     30  sum lines:    202
------------

res: session-3270-num-233-resolution-6         num lines:     25  sum lines:    227
------------

res: session-3270-num-233-resolution-7         num lines:     17  sum lines:    244
------------

res: session-3270-num-233-resolution-8         num lines:      6  sum lines:    250
------------

res: session-3270-num-233-resolution-9         num lines:      3  sum lines:    253
------------

res: s

In [40]:
query = {
    'match': {'metadata.session_id.keyword': session_id}
}
retrieved_resolutions = rep_es.retrieve_resolutions_by_query(query, size=100)
len(retrieved_resolutions)

14

In [28]:
order = [
    'session-3270-num-233-attendance_list',
    'session-3270-num-233-resolution-2',
    'session-3270-num-233-resolution-3',
    'session-3270-num-233-resolution-4',
    'session-3270-num-233-resolution-5',
    'session-3270-num-233-resolution-6',
    'session-3270-num-233-resolution-7',
    'session-3270-num-233-resolution-8',
    'session-3270-num-233-resolution-9',
    'session-3270-num-233-resolution-10',
    'session-3270-num-233-resolution-11',
    'session-3270-num-233-resolution-12',
    'session-3270-num-233-resolution-13',
    'session-3270-num-233-resolution-14',
]
[res.id for res in sorted(retrieved_resolutions, key=lambda r: order.index(r.id))]

['session-3270-num-233-attendance_list',
 'session-3270-num-233-resolution-2',
 'session-3270-num-233-resolution-3',
 'session-3270-num-233-resolution-4',
 'session-3270-num-233-resolution-5',
 'session-3270-num-233-resolution-6',
 'session-3270-num-233-resolution-7',
 'session-3270-num-233-resolution-8',
 'session-3270-num-233-resolution-9',
 'session-3270-num-233-resolution-10',
 'session-3270-num-233-resolution-11',
 'session-3270-num-233-resolution-12',
 'session-3270-num-233-resolution-13',
 'session-3270-num-233-resolution-14']

In [29]:
for res in resolutions:
    print(f"------------\n")
    lines = [line for para in res.paragraphs for tr in para.text_regions for line in tr.lines]
    page_ids = sorted(set(line.metadata['page_id'] for line in lines))
    print(f"{res.id: <40}  {page_ids}\n")
    for para in res.paragraphs:
        print(f"\t{para.text}")
                

------------

session-3270-num-233-attendance_list      ['NL-HaNA_1.01.02_3270_0792-page-1583']

	Preside d'Heer van Gockinga, Present d'Heeren van Heuckelom, Kaldenbach, Ommeren, Johan de Witt, Glas, Raet Pensionnaris de Witt, Reigersbergh, Crommon, Renswoude, Kann, Schrieck, van Lange, Roijer,
------------

session-3270-num-233-resolution-1         ['NL-HaNA_1.01.02_3270_0792-page-1583']

	De resolutien gisteren genomen, sijn gelesen ende geresumeert, gelijck oock geresumeert ende gearresteert sijn de depesches daeruijt resulterende.
------------

session-3270-num-233-resolution-2         ['NL-HaNA_1.01.02_3270_0792-page-1583', 'NL-HaNA_1.01.02_3270_0793-page-1584']

	Ontfangen een Missive vanden Heer Ambassadeur van Gogh, geschreven tot Chelsij den derden deses, gelijck oock een Missive vanden selven date, geaddresseert aenden Griffier Ruijsch, ende daernevens een bijlage: Waerop gedelibereert sijnde, Is goetgevonden ende verstaen, dat de voorschreve leste Missive met de bijlage cop

In [30]:
for res in sorted(retrieved_resolutions, key=lambda r: order.index(r.id)):
    print(f"------------\n")
    lines = [line for para in res.paragraphs for tr in para.text_regions for line in tr.lines]
    page_ids = sorted(set(line.metadata['page_id'] for line in lines))
    print(f"{res.id: <40}  {page_ids}\n")
    for para in res.paragraphs:
        print(f"\t{para.text}")
                

------------

session-3270-num-233-attendance_list      ['NL-HaNA_1.01.02_3270_0792-page-1583']

	Preside d'Heer van Gockinga, Present d'Heeren van Heuckelom, Kaldenbach, Ommeren, Johan de Witt, Glas, Raet Pensionnaris de Witt, Reigersbergh, Crommon, Renswoude, Kann, Schrieck, van Lange, Roijer,
------------

session-3270-num-233-resolution-2         ['NL-HaNA_1.01.02_3270_0792-page-1583']

	De resolutien gisteren genomen, sijn gelesen ende geresumeert, gelijck oock geresumeert ende gearresteert sijn de depesches daeruijt resulterende.
------------

session-3270-num-233-resolution-3         ['NL-HaNA_1.01.02_3270_0792-page-1583', 'NL-HaNA_1.01.02_3270_0793-page-1584']

	Ontfangen een Missive vanden Heer Ambassadeur van Gogh, geschreven tot Chelsij den derden deses, gelijck oock een Missive vanden selven date, geaddresseert aenden Griffier Ruijsch, ende daernevens een bijlage: Waerop gedelibereert sijnde, Is goetgevonden ende verstaen, dat de voorschreve leste Missive met de bijlage cop